## Chaining
We can **chain** transformations and action to create a computation **pipeline**
Suppose we want to compute the sum of the squares
$$ \sum_{i=1}^n x_i^2 $$
where the elements $x_i$ are stored in an RDD.

### Start the `SparkContext`

In [1]:
import numpy as np
from pyspark import SparkContext
sc = SparkContext(master="local[4]")

22/10/07 14:17:56 WARN Utils: Your hostname, 1011000011101110 resolves to a loopback address: 127.0.1.1, but we couldn't find any external IP address!
22/10/07 14:17:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/07 14:17:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/07 14:17:58 WARN MacAddressUtil: Failed to find a usable hardware address from the network interfaces; using random bytes: c1:1d:19:85:01:db:ce:20


In [2]:
B=sc.parallelize(np.random.randint(0,10,size=1000))
lst = B.collect()
for i in lst: 
    print(i,end=', ')

0, 5, 3, 4, 9, 9, 9, 2, 2, 4, 3, 1, 2, 9, 7, 6, 9, 9, 6, 0, 0, 6, 6, 9, 3, 9, 5, 3, 4, 8, 4, 0, 6, 9, 7, 4, 8, 6, 7, 4, 6, 7, 5, 3, 8, 5, 6, 1, 1, 9, 5, 9, 5, 4, 9, 7, 9, 4, 2, 7, 1, 8, 8, 3, 6, 4, 6, 9, 7, 9, 8, 9, 4, 5, 6, 9, 0, 6, 0, 0, 4, 2, 3, 6, 0, 5, 5, 3, 5, 8, 2, 4, 2, 1, 4, 0, 3, 3, 1, 5, 3, 3, 4, 9, 7, 0, 2, 9, 7, 2, 3, 3, 2, 2, 9, 1, 0, 3, 7, 7, 3, 4, 3, 1, 1, 2, 4, 6, 0, 1, 2, 4, 7, 9, 5, 7, 9, 6, 1, 2, 1, 4, 0, 8, 7, 2, 4, 4, 2, 0, 5, 1, 4, 5, 9, 1, 7, 7, 1, 4, 4, 8, 0, 1, 3, 4, 5, 0, 7, 8, 8, 1, 5, 9, 3, 0, 4, 6, 1, 0, 1, 9, 8, 5, 4, 2, 2, 1, 6, 4, 4, 6, 3, 1, 3, 8, 7, 1, 8, 9, 1, 5, 1, 1, 8, 0, 9, 4, 1, 1, 1, 7, 8, 2, 3, 0, 3, 2, 9, 8, 9, 0, 6, 0, 9, 6, 2, 0, 9, 2, 1, 1, 9, 4, 6, 1, 9, 1, 0, 0, 8, 5, 2, 9, 9, 7, 6, 8, 8, 1, 7, 9, 3, 7, 6, 4, 0, 1, 4, 7, 2, 2, 0, 0, 0, 8, 6, 3, 5, 5, 3, 7, 6, 0, 8, 4, 4, 5, 8, 0, 1, 3, 8, 3, 2, 3, 7, 2, 7, 8, 3, 8, 5, 2, 8, 8, 1, 7, 1, 9, 3, 1, 1, 3, 0, 5, 4, 2, 4, 4, 8, 4, 6, 8, 6, 9, 4, 7, 2, 4, 9, 4, 2, 6, 6, 1, 2, 3, 5, 1, 6, 0, 7, 3

### Sequential syntax for chaining
Perform assignment after each computation

In [10]:
%%time
Squares=B.map(lambda x:x*x)
summation = Squares.reduce(lambda x,y:x+y)

CPU times: user 8.17 ms, sys: 4.8 ms, total: 13 ms
Wall time: 179 ms


In [4]:
print(summation)

28168


### Cascaded syntax for chaining
Combine computations into a single cascaded command

In [11]:
%%time
B.map(lambda x:x*x).reduce(lambda x,y:x+y)

CPU times: user 13.9 ms, sys: 190 µs, total: 14.1 ms
Wall time: 176 ms


28168

### Both syntaxes mean exactly the same thing
The only difference:
* In the sequential syntax the intermediate RDD has a name `Squares`
* In the cascaded syntax the intermediate RDD is *anonymous*

The execution is identical!

### Sequential execution
The standard way that the map and reduce are executed is
* perform the map
* store the resulting RDD in memory
* perform the reduce

### Disadvantages of Sequential execution

1. Intermediate result (`Squares`) requires memory space.
2. Two scans of memory (of `B`, then of `Squares`) - double the cache-misses.

### Pipelined execution
Perform the whole computation in a single pass. For each element of **`B`**
1. Compute the square
2. Enter the square as input to the `reduce` operation.

### Advantages of Pipelined execution

1. Less memory required - intermediate result is not stored.
2. Faster - only one pass through the Input RDD.

In [12]:
sc.stop()